In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# DATA

# transforms
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dir = "train"
test_dir = "test"

train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# MODEL
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),  # 32x128x128
            nn.ReLU(),
            nn.MaxPool2d(2),                 
            nn.Conv2d(32, 64, 3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2),                 
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),                 
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 128),
            nn.ReLU(),
            nn.Linear(128, 2)                # 2 classes: cat, dog
        )

    def forward(self, x):
        return self.net(x)

model = SimpleCNN().to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# training loop with validation
for epoch in range(100):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

# EVALUATE on test images
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total

    print(f"Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}, Test Accuracy: {accuracy:.2f}%")

Epoch 1: Train Loss: 0.6899, Test Accuracy: 60.00%
Epoch 2: Train Loss: 0.6742, Test Accuracy: 80.00%
Epoch 3: Train Loss: 0.6409, Test Accuracy: 70.00%
Epoch 4: Train Loss: 0.6202, Test Accuracy: 60.00%
Epoch 5: Train Loss: 0.5834, Test Accuracy: 90.00%
Epoch 6: Train Loss: 0.5631, Test Accuracy: 70.00%
Epoch 7: Train Loss: 0.5572, Test Accuracy: 60.00%
Epoch 8: Train Loss: 0.5097, Test Accuracy: 90.00%
Epoch 9: Train Loss: 0.4832, Test Accuracy: 80.00%
Epoch 10: Train Loss: 0.4718, Test Accuracy: 80.00%
Epoch 11: Train Loss: 0.4427, Test Accuracy: 90.00%
Epoch 12: Train Loss: 0.4347, Test Accuracy: 80.00%
Epoch 13: Train Loss: 0.3943, Test Accuracy: 60.00%
Epoch 14: Train Loss: 0.4060, Test Accuracy: 70.00%
Epoch 15: Train Loss: 0.3697, Test Accuracy: 70.00%
Epoch 16: Train Loss: 0.3576, Test Accuracy: 80.00%
Epoch 17: Train Loss: 0.3075, Test Accuracy: 80.00%
Epoch 18: Train Loss: 0.2994, Test Accuracy: 60.00%
Epoch 19: Train Loss: 0.2600, Test Accuracy: 60.00%
Epoch 20: Train Loss:

In [123]:
# Save the train model and test on internet images

torch.save(model.state_dict(), "cat_dog_classify.pth")

model = SimpleCNN().to(device)
model.load_state_dict(torch.load("cat_dog_classify.pth"))

import os
from PIL import Image
import torch

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Map index to class name from training dataset
idx_to_class = {v: k for k, v in train_dataset.class_to_idx.items()}

internet_dir = "internet"

model.eval()
with torch.no_grad():
    for filename in os.listdir(internet_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(internet_dir, filename)
            image = Image.open(img_path).convert('RGB')  
            
            # apply test transform
            input_tensor = test_transform(image).unsqueeze(0).to(device) 
            
            # predict
            outputs = model(input_tensor)
            _, predicted = torch.max(outputs, 1)
            predicted_label = idx_to_class[predicted.item()]
            
            print(f"File: {filename}, Predicted: {predicted_label}")



File: cat0.jpg, Predicted: Cat
File: cat1.jpg, Predicted: Cat
File: cat2.jpg, Predicted: Cat
File: cat3.jpg, Predicted: Cat
File: cat4.jpg, Predicted: Cat
File: cat5.jpg, Predicted: Cat
File: dog0.jpg, Predicted: Dog
File: dog1.jpg, Predicted: Dog
File: dog2.jpg, Predicted: Dog
File: dog3.jpg, Predicted: Dog
File: dog4.jpg, Predicted: Dog
File: dog5.jpg, Predicted: Dog
